In [7]:
import os
import platform
import pandas as pd
import numpy as np
import time
import asyncio
import json
import itertools
import nest_asyncio
nest_asyncio.apply()
from bs4 import BeautifulSoup

# for logging
import sys
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler

from urllib.request import urlopen
from urllib.parse import urlparse, parse_qs

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

#logging
class PrintLogger:
    def __init__(self, log):
        self.terminal = sys.stdout
        self.log = log

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        pass

current_date = datetime.datetime.now().strftime("%Y-%m-%d")
def setup_logging():
    log_formatter = logging.Formatter("%(asctime)s - %(message)s", "%Y-%m-%d %H:%M:%S")
    log_file = f'verval_scrape_{current_date}.log'
    log_handler = TimedRotatingFileHandler(log_file, when="midnight", interval=1, backupCount=30, utc=False)
    log_handler.setFormatter(log_formatter)
    log_handler.setLevel(logging.DEBUG)
    
    logger = logging.getLogger()
    logger.addHandler(log_handler)

    sys.stdout = PrintLogger(log_handler.stream)
setup_logging()

os_system = platform.system()
print('OS SYSTEM:   ', os_system)

#cpu count
cores = os.cpu_count()
print(f'CPU CORES:    {cores}')

# set path ke file chromedriver to operate the Chrome browser.
chrome_version = 'v116_0_5845_96'
if os_system == 'Windows':
    chrome_path = os.path.join('webdriver', 'chrome', os_system, chrome_version, 'chromedriver.exe')
elif os_system == 'Linux':
    chrome_path = os.path.join('webdriver', 'chrome', os_system, chrome_version, 'chromedriver')
else:
    chrome_path = os.path.join('webdriver', 'chrome', 'MacOS', chrome_version, 'chromedriver')

print('CHROME PATH:    ', chrome_path)
#webdriver options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-setuid-sandbox')
#overcome limited resource problems
# chrome_options.add_argument('--disable-dev-shm-usage')
#open Browser in maximized mode
chrome_options.add_argument("start-maximized")
#disable extension
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

def driversetup():
  # webdriver_service = ChromeService(ChromeDriverManager().install())
  chrome_service = Service(executable_path=chrome_path)
  driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
  return driver

driver = driversetup()

#create empty dataframe
df_rekapitulasi = df_provinsi = df_kabupaten = df_kecamatan = df_sekolah_id_nama = pd.DataFrame()
empty_series = pd.Series()

kode_wilayah_kabupaten = pd.Series([])
kode_wilayah_kecamatan = pd.Series([])
sekolah_id_enkrip = pd.Series([])
semester_label = pd.Series([])
semester_period = pd.Series([])

#for append link string
base_url = 'https://dapo.kemdikbud.go.id'

province_api = 'https://dapo.kemdikbud.go.id/rekap/dataPD?id_level_wilayah=0&kode_wilayah=000000&semester_id=20222'
provinces = pd.read_json(province_api)

provinces = provinces.sort_values('kode_wilayah')
provinces = provinces['kode_wilayah']
provinces = provinces.astype(str).str.zfill(6)

# """## get kode wilayah `kabupaten`"""
# start = time.time()
# print('hi')
# for province in provinces[0:1]:
#     province = province.strip()
#     kab_api = 'https://dapo.kemdikbud.go.id/rekap/dataPD?id_level_wilayah=1&kode_wilayah={0}&semester_id=20231'.format(province)

#     ##Get kode_wilayah kecamatan list dari response api level 2
#     kabupatens = pd.read_json(kab_api)
#     kabupatens = kabupatens.sort_values('kode_wilayah')
#     kabupatens = kabupatens['kode_wilayah']
#     kode_wilayah_kabupaten = kode_wilayah_kabupaten._append(kabupatens, ignore_index=True)

# kode_wilayah_kabupaten = kode_wilayah_kabupaten.astype(str).str.zfill(6)
# end = time.time()
# print('time:', end-start)

C:\Users\steva\AppData\Local\Temp\ipykernel_15304\2122112512.py:94: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  empty_series = pd.Series()
C:\Users\steva\AppData\Local\Temp\ipykernel_15304\2122112512.py:96: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  kode_wilayah_kabupaten = pd.Series([])
C:\Users\steva\AppData\Local\Temp\ipykernel_15304\2122112512.py:97: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  kode_wilayah_kecamatan = pd.Series([])
C:\Users\steva\AppData\Local\Temp\ipykernel_15304\2122112512.py:98: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitl